In [1]:

from utils_aa import * 

study_real = optuna.create_study(
    study_name='study_real',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True, 
)


study_real_sampling = optuna.create_study(
    study_name='study_real_sampling',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)



make_pareto_plot(study_real_sampling)
make_ranking_plots(study_real_sampling, get_median_dict(study_real), data ="_best_artifact/optuna/data.csv", CUTOFF_AUC =.9, CUTOFF_PHE=.0)

[I 2023-05-03 21:07:29,645] Using an existing study with name 'study_real' instead of creating a new one.
[I 2023-05-03 21:07:29,663] Using an existing study with name 'study_real_sampling' instead of creating a new one.


{'best_iteration': '39', 'best_ntree_limit': '40', 'best_score': '0.08114367102583249'}
Phe_value :  0.15181865 	 Phe_ranking :  15.0


{'best_iteration': '2', 'best_ntree_limit': '3', 'best_score': '0.42957672476768494'}
Phe_value :  0.21201801 	 Phe_ranking :  8.0


{'best_iteration': '20', 'best_ntree_limit': '21', 'best_score': '0.21961123868823051'}
Phe_value :  0.25939038 	 Phe_ranking :  7.0


{'best_iteration': '22', 'best_ntree_limit': '23', 'best_score': '0.3154486215983828'}
Phe_value :  0.30960056 	 Phe_ranking :  5.0


{'best_iteration': '14', 'best_ntree_limit': '15', 'best_score': '0.35881624184548855'}
Phe_value :  0.22882034 	 Phe_ranking :  9.0


{'best_iteration': '11', 'best_ntree_limit': '12', 'best_score': '0.4599017488459746'}
Phe_value :  0.36585155 	 Phe_ranking :  6.0


{'best_iteration': '25', 'best_ntree_limit': '26', 'best_score': '0.1610239134170115'}
Phe_value :  0.25733283 	 Phe_ranking :  10.0


{'best_iteration': '4', 'best_ntree_limit': '5', 'best_sco

# ----------------------------------------------------

In [ ]:

#study_name: str = "alejandro_resampled_data_BorderlineSMOTE"

study_BorderlineSMOTE = optuna.create_study(
    study_name='study_BorderlineSMOTE',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)

study_SVMSMOTE = optuna.create_study(
    study_name='study_SVMSMOTE',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)


study_ADASYN = optuna.create_study(
    study_name='study_ADASYN',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)

study_real = optuna.create_study(
    study_name='study_real',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True, 
)

# ########################
# study_real.optimize(lambda trial: objective(trial, "_best_artifact/optuna/data.csv",                                                        finetunning = True), n_trials=5000, n_jobs=-1)
# study_BorderlineSMOTE.optimize(lambda trial: objective(trial, "_best_artifact/optuna/resampled_data_BorderlineSMOTE.csv",                   finetunning = True), n_trials=5000, n_jobs=-1)
# study_SVMSMOTE.optimize(lambda trial: objective(trial, "_best_artifact/optuna/resampled_data_SVMSMOTE.csv",                                 finetunning = True), n_trials=5000, n_jobs=-1)
# study_ADASYN.optimize(lambda trial: objective(trial, "_best_artifact/optuna/resampled_data_ADASYN.csv",                                     finetunning = True), n_trials=5000, n_jobs=-1)
# ###################
#make_pareto_plot(study_real)


In [2]:

######## SAMPLING ########

study_BorderlineSMOTE_sampling = optuna.create_study(
    study_name='study_BorderlineSMOTE_sampling',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)

study_SVMSMOTE_sampling = optuna.create_study(
    study_name='study_SVMSMOTE_sampling',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)



study_ADASYN_sampling = optuna.create_study(
    study_name='study_ADASYN_sampling',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)

study_real_sampling = optuna.create_study(
    study_name='study_real_sampling',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)


[I 2023-05-03 20:49:37,279] Using an existing study with name 'study_BorderlineSMOTE_sampling' instead of creating a new one.
[I 2023-05-03 20:49:37,296] Using an existing study with name 'study_SVMSMOTE_sampling' instead of creating a new one.
[I 2023-05-03 20:49:37,314] Using an existing study with name 'study_ADASYN_sampling' instead of creating a new one.
[I 2023-05-03 20:49:37,330] Using an existing study with name 'study_real_sampling' instead of creating a new one.


In [3]:



# study_real_sampling.optimize(lambda trial: objective(trial, "_best_artifact/optuna/data.csv",  tuned_params = get_median_dict(study_real),                    finetunning = False), n_trials=1000, n_jobs=-1)
# study_ADASYN_sampling.optimize(lambda trial: objective(trial, "_best_artifact/optuna/resampled_data_ADASYN.csv",  tuned_params = get_median_dict(study_ADASYN),                    finetunning = False), n_trials=1000, n_jobs=-1)
# study_BorderlineSMOTE_sampling.optimize(lambda trial: objective(trial, "_best_artifact/optuna/resampled_data_BorderlineSMOTE.csv",  tuned_params = get_median_dict(study_BorderlineSMOTE),  finetunning = False), n_trials=1000, n_jobs=-1)
# study_SVMSMOTE_sampling.optimize(lambda trial: objective(trial, "_best_artifact/optuna/resampled_data_SVMSMOTE.csv",  tuned_params = get_median_dict(study_SVMSMOTE),                finetunning = False), n_trials=1000, n_jobs=-1)




In [ ]:
make_pareto_plot(study_real_sampling)
make_ranking_plots(study_real_sampling, get_median_dict(study_real), data ="_best_artifact/optuna/data.csv", CUTOFF_AUC =.9, CUTOFF_PHE=.0)

In [ ]:
make_pareto_plot(study_ADASYN_sampling)
make_ranking_plots(study_ADASYN_sampling, get_median_dict(study_ADASYN), "_best_artifact/optuna/resampled_data_ADASYN.csv", CUTOFF_AUC =.9, CUTOFF_PHE=.3)

In [ ]:
make_pareto_plot(study_BorderlineSMOTE_sampling)
make_ranking_plots(study_BorderlineSMOTE_sampling, get_median_dict(study_BorderlineSMOTE),data="_best_artifact/optuna/resampled_data_BorderlineSMOTE.csv" ,CUTOFF_AUC =.9, CUTOFF_PHE=.3)



In [ ]:

make_pareto_plot(study_SVMSMOTE_sampling)
make_ranking_plots(study_SVMSMOTE_sampling, get_median_dict(study_SVMSMOTE), data="_best_artifact/optuna/resampled_data_SVMSMOTE.csv",CUTOFF_AUC =.9, CUTOFF_PHE=.1)